# Transmission DB Seed

This notebook imports a JSON object describing parts in the `transmission`, their `supplier(s)` and aomne compatibility relationships.

In [58]:
# Dependencies and imports
!pip install lorem-text

from lorem_text import lorem
from yfiles_jupyter_graphs import GraphWidget
import random


In [59]:
# Connect to DB
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://89698250.databases.neo4j.io"
AUTH = ("neo4j", "nN4v9Y33RjQtEPzqgNAUUbKQj6Os9Hbs_AuGSY7s1DQ")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [60]:
# Clear the DB - Uncomment to use

records, summary, keys = driver.execute_query(
    "MATCH (n) DETACH DELETE (n);",
    database_="neo4j",
)

In [65]:
# Create Transmission Parts and supplier relationships

# transmission
# 2 crank + 2 chain

# crank: weight, material, cost, durability (in kms), width (crank), teeth-width
# chain: weight, material, cost, durability (in kms), width (crank), teeth-width, total_length


# self dynamic attributes - relationships: weight x weight, weight x material, weight....... fractional value 0-2 (123%) or null


# relationship: crank x chain
#	is_compatible_manufacturer
# 	is_compatible_material (material == material)
# 	is_compatible_teeth (teethxwidth == teethxwidth)

# 	is_durability_transmission - min(durability crank, dur chain)   - 12 (km)



import json

parts = [
    {
        "sku": "FC-M9020-B1",
        "name": "XTR HOLLOWTECH II MTB Crankset 174mm Q-Factor 1x11-speed",
        "series": "XTR M9000 Series",
        "brand": "Shimano",
        "type": "Crankset",
        "weight": 593,
        "width": 6,
        "teeth_spacing": 8,
        "material": "titanium/steel",
        "cost": 54535,
        "durability": 300,
        "compatible_with":
        [
            "HG-X11"
        ],
        "compatible_materials": [],
        "compatible_teeth_spacing": [],
        "durability_transmission": []
    },
    {
        "sku": "HG-X11",
        "name": "HG-X11 Chain",
        "series": "XTR M9000 Series",
        "brand": "Shimano",
        "type": "Chain",
        "weight": 247,
        "cost": 3932,
        "material": "steel",
        "durability": 200,
        "width": 6,
        "teeth_spacing": 8,
        "compatible_with":
        [
            "FC-M9020-B1"
        ],
        "compatible_materials": [],
        "compatible_teeth_spacing": [],
        "durability_transmission": []
    },
    {
        "sku": "FC-4700",
        "name": "SHIMANO TIAGRA Road Crankset 2x10-speed",
        "series": "TIAGRA 4700 Series",
        "brand": "Shimano",
        "type": "Crankset",
        "weight": 667,
        "cost": 9299,
        "material": "steel",
        "durability": 250,
        "width": 6,
        "teeth_spacing": 7,
        "compatible_with":
        [
            "CN-HG54"
        ],
        "compatible_materials": [],
        "compatible_teeth_spacing": [],
        "durability_transmission": []
    },
    {
        "sku": "CN-HG54",
        "name": "SHIMANO DEORE 10-Speed Super Narrow MTB Chain",
        "series": "GRX 10-speed",
        "brand": "Shimano",
        "type": "Chain",
        "weight": 273,
        "cost": 1703,
        "material": "steel",
        "durability": 150,
        "width": 6,
        "teeth_spacing": 7,
        "compatible_with":
        [
            "FC-4700"
        ],
        "compatible_materials": [],
        "compatible_teeth_spacing": [],
        "durability_transmission": []
    }
]

keysToLink = ["weight", "material", "cost", "durability", "width", "teeth_spacing"]

partsCopy = []
random.seed

# self dynamic attributes
for part in parts:
    pc = part.copy()
    for attr in part:
        # print(attr)
        # print(part[attr])
        if attr in keysToLink:
            for key in keysToLink:
                # print(attr + "_x_" + key)
                if attr == key:
                    pc[attr + "_x_" + key] = 100
                else:
                    pc[attr + "_x_" + key] = random.randint(1, 200)
                
    partsCopy.append(pc)

# print(partsCopy)

partsWithRelationShips = []
for part in partsCopy:
    pc = part.copy()
    for relationPart in partsCopy:
        if part['sku'] != relationPart['sku']:
            if part['material'] == relationPart['material']:
                pc['compatible_materials'].append(relationPart["sku"])

            if part['teeth_spacing'] == relationPart['teeth_spacing']:
                pc['compatible_teeth_spacing'].append(relationPart["sku"])

            if part['type'] == "Crankset":
                if relationPart['type'] == "Chain":
                    pc['durability_transmission'].append({"sku":relationPart["sku"],"durability":min(part['durability'],relationPart['durability'])})
            elif part['type'] == "Chain":
                    pc['durability_transmission'].append({"sku":relationPart["sku"],"durability":min(part['durability'],relationPart['durability'])})

    partsWithRelationShips.append(pc)
                
# print(partsWithRelationShips)

# # Serializing json
json_object = json.dumps(partsWithRelationShips, indent=4)
 
# # Writing to sample.json
with open("parts.json", "w") as outfile:
    outfile.write(json_object)

cypher_query = """
WITH $json as data
UNWIND data AS p 
MERGE (part:TransmissionPart {sku:p.sku}) SET
	part.name = p.name, 
	part.brand = p.brand, 
	part.series = p.series, 
	part.type = p.type, 
	part.weight = p.weight, 
	part.material = p.material, 
	part.durability = p.durability,
	part.width = p.width, 
	part.teeth_spacing = p.teeth_spacing,
	part.weight_x_weight = p.weight_x_weight,
	part.weight_x_material = p.weight_x_material,
	part.weight_x_cost = p.weight_x_cost,
	part.weight_x_durability = p.weight_x_durability,
	part.weight_x_width = p.weight_x_width,
	part.weight_x_teeth_spacing = p.weight_x_teeth_spacing,
	part.width_x_weight = p.width_x_weight,
	part.width_x_material = p.width_x_material,
	part.width_x_cost = p.width_x_cost,
	part.width_x_durability = p.width_x_durability,
	part.width_x_width = p.width_x_width,
	part.width_x_teeth_spacing = p.width_x_teeth_spacing,
	part.teeth_spacing_x_weight = p.teeth_spacing_x_weight,
	part.teeth_spacing_x_material = p.teeth_spacing_x_material,
	part.teeth_spacing_x_cost = p.teeth_spacing_x_cost,
	part.teeth_spacing_x_durability = p.teeth_spacing_x_durability,
	part.teeth_spacing_x_width = p.teeth_spacing_x_width,
	part.teeth_spacing_x_teeth_spacing = p.teeth_spacing_x_teeth_spacing,
	part.material_x_weight = p.material_x_weight,
	part.material_x_material = p.material_x_material,
	part.material_x_cost = p.material_x_cost,
	part.material_x_durability = p.material_x_durability,
	part.material_x_width = p.material_x_width,
	part.material_x_teeth_spacing = p.material_x_teeth_spacing,
	part.cost_x_weight = p.cost_x_weight,
	part.cost_x_material = p.cost_x_material,
	part.cost_x_cost = p.cost_x_cost,
	part.cost_x_durability = p.cost_x_durability,
	part.cost_x_width = p.cost_x_width,
	part.cost_x_teeth_spacing = p.cost_x_teeth_spacing,
	part.durability_x_weight = p.durability_x_weight,
	part.durability_x_material = p.durability_x_material,
	part.durability_x_cost = p.durability_x_cost,
	part.durability_x_durability = p.durability_x_durability,
	part.durability_x_width = p.durability_x_width,
	part.durability_x_teeth_spacing = p.durability_x_teeth_spacing

FOREACH (cw IN p.compatible_with | 
	MERGE (p2:TransmissionPart {sku: cw}) 
	MERGE (part)-[:IS_COMPATIBLE_MANUFACTURER]->(p2)
 	MERGE (p2)-[:IS_COMPATIBLE_MANUFACTURER]->(part))

FOREACH (cw IN p.compatible_materials | 
	MERGE (p2:TransmissionPart {sku: cw}) 
	MERGE (part)-[:IS_COMPATIBLE_MATERIAL]->(p2)
 	MERGE (p2)-[:IS_COMPATIBLE_MATERIAL]->(part))

FOREACH (cw IN p.compatible_teeth_spacing | 
	MERGE (p2:TransmissionPart {sku: cw}) 
	MERGE (part)-[:IS_COMPATIBLE_TEETH_SPACING]->(p2)
 	MERGE (p2)-[:IS_COMPATIBLE_TEETH_SPACING]->(part))

FOREACH (cw IN p.durability_transmission | 
	MERGE (p2:TransmissionPart {sku: cw.sku}) 
	MERGE (part)-[:IS_DURABILITY_TRANSMISSION {durability: cw.durability}]->(p2)
 	MERGE (p2)-[:IS_DURABILITY_TRANSMISSION {durability: cw.durability}]->(part))

"""

with driver.session() as session:
    result = session.run(cypher_query, json = partsWithRelationShips)

print("Done")
    


Done


In [66]:
# Query DB for parts and relationships

with driver.session(database="neo4j") as session:
  graph = session.run("MATCH (p:TransmissionPart)-[r]->(a) RETURN p,r,a;").graph()

# graph


In [78]:
# custom graph functions

def custom_node_color_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "TransmissionPart":
        if node["properties"]["type"] == "Crankset":
            return "green"
        elif node["properties"]["type"] == "Chain":
            return "orange"
    
def custom_node_label_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "TransmissionPart":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]

def custom_edge_label_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "IS_COMPATIBLE_MANUFACTURER":
        return "blue"
    elif node["properties"]["label"] == "IS_COMPATIBLE_TEETH_SPACING":
        return "purple"
    elif node["properties"]["label"] == "IS_COMPATIBLE_MATERIAL":
        return "red"
    else:
        return "black"


In [79]:
# Render the graph...

w = GraphWidget(graph=graph)
w.directed = False
w.set_graph_layout("orthogonal")
# w.set_neighborhood(1,[graph.nodes.get(0).id])
w.set_sidebar(enabled = False, start_with = "Neighbourhood")
w.set_overview(False)

w.set_node_color_mapping(custom_node_color_mapping)
w.set_edge_color_mapping(custom_edge_label_mapping)
w.set_node_label_mapping(custom_node_label_mapping)

w.show()


GraphWidget(layout=Layout(height='500px', width='100%'))